In [103]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import pickle
import datetime
import platform
import gala
import astropy
from astropy.coordinates import CartesianRepresentation, CartesianDifferential
from sklearn.decomposition import PCA
from scipy.ndimage import uniform_filter1d


from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D


from gala.units import galactic
from gala.potential import Hamiltonian
from gala.potential import LogarithmicPotential
from gala.dynamics import PhaseSpacePosition
from gala.dynamics.mockstream import (
    MockStreamGenerator,
    FardalStreamDF
)
from gala.integrate import LeapfrogIntegrator


from tqdm.notebook import tqdm
import time
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter

In [2]:
with open("../data/gc_stream_ensemble.pkl", "rb") as f:
    data = pickle.load(f)

streams = data["streams"]

In [3]:
streams

[{'halo': 'spherical',
  'q': 1.0,
  'progenitor_id': 0,
  'mass': <Quantity 8000. solMass>,
  'stream': (<MockStream cartesian, dim=3, shape=(12000000,)>,
   <PhaseSpacePosition cartesian, dim=3, shape=(1,)>),
  'orbit': <Orbit cartesian, dim=3, shape=(4000,)>,
  't': <Quantity [ 0.000e+00, -1.000e+00, -2.000e+00, ..., -3.997e+03, -3.998e+03,
             -3.999e+03] Myr>},
 {'halo': 'spherical',
  'q': 1.0,
  'progenitor_id': 1,
  'mass': <Quantity 10000. solMass>,
  'stream': (<MockStream cartesian, dim=3, shape=(12000000,)>,
   <PhaseSpacePosition cartesian, dim=3, shape=(1,)>),
  'orbit': <Orbit cartesian, dim=3, shape=(4000,)>,
  't': <Quantity [ 0.000e+00, -1.000e+00, -2.000e+00, ..., -3.997e+03, -3.998e+03,
             -3.999e+03] Myr>},
 {'halo': 'spherical',
  'q': 1.0,
  'progenitor_id': 2,
  'mass': <Quantity 20000. solMass>,
  'stream': (<MockStream cartesian, dim=3, shape=(12000000,)>,
   <PhaseSpacePosition cartesian, dim=3, shape=(1,)>),
  'orbit': <Orbit cartesian, di

In [4]:
int(streams[0]['mass'].value)

8000

## Configuration space 3D movie :

In [34]:
def animate_stream_3d(
    stream,
    orbit,
    t,
    filename="stream_motion.mp4",
    interval=50,
    trail_length=25,
    n_stream_viz=800
):
    """
    Physically honest, fast animation:
    - Static present-day stream
    - Moving progenitor orbit
    - Orbit trail shows time evolution
    """

    # --- Stream phase-space (present-day snapshot) ---
    sx, sy, sz = stream.pos.xyz.to_value(u.kpc)
    svx, svy, svz = stream.vel.d_xyz.to_value(u.km/u.s)
    speed = np.sqrt(svx**2 + svy**2 + svz**2)

    # Subsample for visualization only
    if len(sx) > n_stream_viz:
        idx = np.random.choice(len(sx), n_stream_viz, replace=False)
        sx, sy, sz = sx[idx], sy[idx], sz[idx]
        speed = speed[idx]

    # --- Orbit ---
    ox, oy, oz = orbit.pos.xyz.to_value(u.kpc)

    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection="3d")

    lim = 20
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(-lim, lim)

    ax.set_xlabel("X [kpc]")
    ax.set_ylabel("Y [kpc]")
    ax.set_zlabel("Z [kpc]")

    # --- Static full orbit (context) ---
    ax.plot(
        ox, oy, oz,
        color="black",
        linestyle="dotted",
        lw=0.5,
        zorder=-1
    )

    # --- Static stream (draw ONCE) ---
    sc = ax.scatter(
        sx, sy, sz,
        c=speed,
        cmap="viridis",
        s=2,
        alpha=0.6,
        antialiased=False
    )

    plt.colorbar(sc, ax=ax, label="Speed [km/s]", pad=0.1)

    # --- Animated orbit segment ---
    orbit_line, = ax.plot([], [], [], lw=2.5, color="crimson")

    def update(frame):
        start = max(0, frame - trail_length)

        orbit_line.set_data(
            ox[start:frame], oy[start:frame]
        )
        orbit_line.set_3d_properties(
            oz[start:frame]
        )

        ax.set_title(f"t = {t[frame].to_value(u.Myr):.0f} Myr")
        return orbit_line,

    frame_stride = 40
    frames = range(0, len(t), frame_stride)

    ani = FuncAnimation(
        fig,
        update,
        frames=frames,
        interval=interval,
        blit=False
    )

    writer = FFMpegWriter(
        fps=10,
        codec="libx264",
        bitrate=1500,
        extra_args=["-preset", "fast"]
    )

    ani.save(filename, writer=writer, dpi=120)
    plt.close(fig)

    print(f"Saved animation to {filename}")

In [35]:
for i, s in enumerate(streams):
    
    prog_mass = int(s['mass'].value)
    
    animate_stream_3d(
        stream=s["stream"][0],
        orbit=s["orbit"],
        t=s["t"],
        filename=f"../figures/stream_{prog_mass}_{s['halo']}.mp4"
    )

Saved animation to ../figures/stream_8000_spherical.mp4
Saved animation to ../figures/stream_10000_spherical.mp4
Saved animation to ../figures/stream_20000_spherical.mp4
Saved animation to ../figures/stream_8000_oblate.mp4
Saved animation to ../figures/stream_10000_oblate.mp4
Saved animation to ../figures/stream_20000_oblate.mp4
Saved animation to ../figures/stream_8000_prolate.mp4
Saved animation to ../figures/stream_10000_prolate.mp4
Saved animation to ../figures/stream_20000_prolate.mp4


## Phase-Space movie

In [117]:
def extract_stream_snapshot(stream_tuple, t_array, time_index=0):

    mock = stream_tuple[0]
    nt = len(t_array)

    total_points = mock.pos.x.shape[0]

    if total_points % nt != 0:
        raise ValueError(
            "Total points not divisible by number of time steps."
        )

    npart = total_points // nt

    # Reshape to (nt, npart)
    x_all = mock.pos.x.reshape(nt, npart)
    y_all = mock.pos.y.reshape(nt, npart)
    z_all = mock.pos.z.reshape(nt, npart)

    vx_all = mock.vel.d_x.reshape(nt, npart)
    vy_all = mock.vel.d_y.reshape(nt, npart)
    vz_all = mock.vel.d_z.reshape(nt, npart)

    # IMPORTANT: Reverse time axis if needed
    # Because stream storage is often reversed relative to t_array
    x_all = x_all[::-1]
    y_all = y_all[::-1]
    z_all = z_all[::-1]

    vx_all = vx_all[::-1]
    vy_all = vy_all[::-1]
    vz_all = vz_all[::-1]

    # Extract requested epoch
    x = x_all[time_index]
    y = y_all[time_index]
    z = z_all[time_index]

    vx = vx_all[time_index]
    vy = vy_all[time_index]
    vz = vz_all[time_index]

    pos = CartesianRepresentation(x, y, z)
    vel = CartesianDifferential(vx, vy, vz)

    pos = pos.with_differentials(vel)

    return PhaseSpacePosition(pos)

In [63]:
def make_galactic_hamiltonian(q=1.0):
    pot = LogarithmicPotential(
        v_c=220 * u.km/u.s,
        r_h=12 * u.kpc,
        q1=1.0,
        q2=1.0,
        q3=q,
        units=galactic
    )
    return Hamiltonian(pot)

In [139]:
def animate_full_phase_space(
    stream_orbits,
    orbit,
    t,
    halo_label="",
    mass_label="",
    filename="full_phase_space.mp4",
    n_stream_viz=1000
):

    # ------------------------------
    # Extract arrays
    # ------------------------------

    sx = stream_orbits.pos.x.to_value(u.kpc)
    sy = stream_orbits.pos.y.to_value(u.kpc)
    sz = stream_orbits.pos.z.to_value(u.kpc)

    svx = stream_orbits.vel.d_x.to_value(u.km/u.s)
    svy = stream_orbits.vel.d_y.to_value(u.km/u.s)
    svz = stream_orbits.vel.d_z.to_value(u.km/u.s)

    ox = orbit.pos.x.to_value(u.kpc)
    oy = orbit.pos.y.to_value(u.kpc)
    oz = orbit.pos.z.to_value(u.kpc)

    ovx = orbit.vel.d_x.to_value(u.km/u.s)
    ovy = orbit.vel.d_y.to_value(u.km/u.s)
    ovz = orbit.vel.d_z.to_value(u.km/u.s)

    # Subsample
    if sx.shape[1] > n_stream_viz:
        idx = np.random.choice(sx.shape[1], n_stream_viz, replace=False)
        sx, sy, sz = sx[:, idx], sy[:, idx], sz[:, idx]
        svx, svy, svz = svx[:, idx], svy[:, idx], svz[:, idx]

    # ------------------------------
    # Global Δv normalization
    # ------------------------------

    dv_all = svx - ovx[:, None]
    dv_max = np.percentile(np.abs(dv_all), 95)
    norm = Normalize(vmin=-dv_max, vmax=dv_max)

    # ------------------------------
    # Precompute misalignment
    # ------------------------------

    misalignment = np.zeros(len(t))

    for i in range(len(t)):
        stream_xyz = np.vstack([sx[i], sy[i], sz[i]]).T
        prog_vel = np.array([ovx[i], ovy[i], ovz[i]])

        pca = PCA(n_components=1)
        pca.fit(stream_xyz)
        axis = pca.components_[0]
        axis /= np.linalg.norm(axis)

        vhat = prog_vel / np.linalg.norm(prog_vel)

        cosang = np.clip(np.abs(np.dot(axis, vhat)), 0, 1)
        misalignment[i] = np.degrees(np.arccos(cosang))

    # Smooth
    misalignment_smooth = uniform_filter1d(misalignment, size=5)

    # ------------------------------
    # Setup figure
    # ------------------------------

    fig, axs = plt.subplots(2, 2, figsize=(15.3, 13.5))
    
    ax_x, ax_y, ax_z, ax_ang = axs.flatten()

    # Consistent axis limits
    ax_x.set_xlim(sx.min(), sx.max())
    ax_x.set_ylim(svx.min(), svx.max())

    ax_y.set_xlim(sy.min(), sy.max())
    ax_y.set_ylim(svy.min(), svy.max())

    ax_z.set_xlim(sz.min(), sz.max())
    ax_z.set_ylim(svz.min(), svz.max())

    ax_x.set_xlabel("x [kpc]")
    ax_x.set_ylabel("v_x [km/s]")
    ax_y.set_xlabel("y [kpc]")
    ax_y.set_ylabel("v_y [km/s]")
    ax_z.set_xlabel("z [kpc]")
    ax_z.set_ylabel("v_z [km/s]")

    # Static orbit
    ax_x.plot(ox, ovx, "k:", lw=0.5)
    ax_y.plot(oy, ovy, "k:", lw=0.5)
    ax_z.plot(oz, ovz, "k:", lw=0.5)

    # Scatter
    sc_x = ax_x.scatter([], [], s=6, cmap="coolwarm", norm=norm)
    sc_y = ax_y.scatter([], [], s=6, cmap="coolwarm", norm=norm)
    sc_z = ax_z.scatter([], [], s=6, cmap="coolwarm", norm=norm)

    # Compute symmetric normalization per component
    v_c = 220.0  # km/s

    dvx_all = (svx - ovx[:, None]) / v_c
    dvy_all = (svy - ovy[:, None]) / v_c
    dvz_all = (svz - ovz[:, None]) / v_c
    
    # dvx_max = np.percentile(np.abs(dvx_all), 95)
    # dvy_max = np.percentile(np.abs(dvy_all), 95)
    # dvz_max = np.percentile(np.abs(dvz_all), 95)

    dvx_max = 0.1
    dvy_max = 0.1
    dvz_max = 0.1
    
    norm_x = Normalize(vmin=-dvx_max, vmax=dvx_max)
    norm_y = Normalize(vmin=-dvy_max, vmax=dvy_max)
    norm_z = Normalize(vmin=-dvz_max, vmax=dvz_max)
    
    sc_x.set_norm(norm_x)
    sc_y.set_norm(norm_y)
    sc_z.set_norm(norm_z)
    
    # -------- X panel colorbar --------
    divider_x = make_axes_locatable(ax_x)
    cax_x = divider_x.append_axes("top", size="5%", pad=0.6)
    
    cbar_x = fig.colorbar(sc_x, cax=cax_x, orientation="horizontal",)
    cbar_x.set_label(r"$\Delta v_x / v_c$ [km s$^{-1}$]")
    cbar_x.set_ticks([-dvx_max, 0, dvx_max])
    
    # -------- Y panel colorbar --------
    divider_y = make_axes_locatable(ax_y)
    cax_y = divider_y.append_axes("top", size="5%", pad=0.6)
    
    cbar_y = fig.colorbar(sc_y, cax=cax_y, orientation="horizontal")
    cbar_y.set_label(r"$\Delta v_y / v_c$ [km s$^{-1}$]")
    cbar_y.set_ticks([-dvy_max, 0, dvy_max])
    
    # -------- Z panel colorbar --------
    divider_z = make_axes_locatable(ax_z)
    cax_z = divider_z.append_axes("top", size="5%", pad=0.6)
    
    cbar_z = fig.colorbar(sc_z, cax=cax_z, orientation="horizontal")
    cbar_z.set_label(r"$\Delta v_z / v_c$ [km s$^{-1}$]")
    cbar_z.set_ticks([-dvz_max, 0, dvz_max])

    # Progenitor
    prog_x, = ax_x.plot([], [], "ko")
    prog_y, = ax_y.plot([], [], "ko")
    prog_z, = ax_z.plot([], [], "ko")

    # Axis/tangent lines
    axis_line_x, = ax_x.plot([], [], "r-", lw=2, label='Stream principal axis (PCA)')
    axis_line_y, = ax_y.plot([], [], "r-", lw=2, label='Stream principal axis (PCA)')
    axis_line_z, = ax_z.plot([], [], "r-", lw=2, label='Stream principal axis (PCA)')

    tangent_line_x, = ax_x.plot([], [], "b-", lw=2, label='Orbit tangent (velocity direction)')
    tangent_line_y, = ax_y.plot([], [], "b-", lw=2, label='Orbit tangent (velocity direction)')
    tangent_line_z, = ax_z.plot([], [], "b-", lw=2, label='Orbit tangent (velocity direction)')

    
    # Misalignment panel
    ax_ang.set_xlabel("Lookback time [Gyr]")
    ax_ang.set_ylabel("Misalignment angle [deg]")
    ax_ang.set_xlim(-t[0].to_value(u.Gyr), -t[-1].to_value(u.Gyr))
    ax_ang.set_ylim(0, misalignment.max()*1.1)

    mis_line, = ax_ang.plot([], [], color="purple")

    angle_text = fig.text(
    0.65, 0.93, "",
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none')
    )

    # ------------------------------
    # Legend (generic, static)
    # ------------------------------

    legend_elements = [
        Line2D([0], [0], color='blue', lw=2,
               label='Orbit tangent'),
        Line2D([0], [0], color='red', lw=2,
               label='Stream principal axis (PCA)')
    ]
    
    # Place legend in first panel (x–vx)
    fig.legend(
        handles=legend_elements,
        loc="upper center",
        bbox_to_anchor=(0.5, 0.5),
        ncol=2,
        frameon=True,
        framealpha=0.9,
        edgecolor='black',
        fontsize=10
    )

    # ------------------------------
    # Update
    # ------------------------------

    def update(frame):

        dvx = (svx[frame] - ovx[frame]) / v_c
        dvy = (svy[frame] - ovy[frame]) / v_c
        dvz = (svz[frame] - ovz[frame]) / v_c

        sc_x.set_offsets(np.c_[sx[frame], svx[frame]])
        sc_x.set_array(dvx)

        sc_y.set_offsets(np.c_[sy[frame], svy[frame]])
        sc_y.set_array(svy[frame] - ovy[frame])

        sc_z.set_offsets(np.c_[sz[frame], svz[frame]])
        sc_z.set_array(svz[frame] - ovz[frame])

        prog_x.set_data([ox[frame]], [ovx[frame]])
        prog_y.set_data([oy[frame]], [ovy[frame]])
        prog_z.set_data([oz[frame]], [ovz[frame]])

        # ----------------------------------
        # Compute principal axis (3D)
        # ----------------------------------
        stream_xyz = np.vstack([sx[frame], sy[frame], sz[frame]]).T
        
        pca = PCA(n_components=1)
        pca.fit(stream_xyz)
        
        axis = pca.components_[0]
        axis /= np.linalg.norm(axis)
        
        # ----------------------------------
        # Normalize orbit velocity (3D)
        # ----------------------------------
        vhat = np.array([ovx[frame], ovy[frame], ovz[frame]])
        vhat /= np.linalg.norm(vhat)
        
        # ----------------------------------
        # FIX: enforce consistent sign
        # ----------------------------------
        if np.dot(axis, vhat) < 0:
            axis = -axis
        
        # Visualization scale (spatial only)
        scale = 0.15 * (ax_x.get_xlim()[1] - ax_x.get_xlim()[0])
        
        # ----------------------------------
        # X–Vx panel (project onto x)
        # ----------------------------------
        axis_line_x.set_data(
            [ox[frame], ox[frame] + scale * axis[0]],
            [ovx[frame], ovx[frame]]
        )
        
        tangent_line_x.set_data(
            [ox[frame], ox[frame] + scale * vhat[0]],
            [ovx[frame], ovx[frame]]
        )
        
        # ----------------------------------
        # Y–Vy panel (project onto y)
        # ----------------------------------
        axis_line_y.set_data(
            [oy[frame], oy[frame] + scale * axis[1]],
            [ovy[frame], ovy[frame]]
        )
        
        tangent_line_y.set_data(
            [oy[frame], oy[frame] + scale * vhat[1]],
            [ovy[frame], ovy[frame]]
        )
        
        # ----------------------------------
        # Z–Vz panel (project onto z)
        # ----------------------------------
        axis_line_z.set_data(
            [oz[frame], oz[frame] + scale * axis[2]],
            [ovz[frame], ovz[frame]]
        )
        
        tangent_line_z.set_data(
            [oz[frame], oz[frame] + scale * vhat[2]],
            [ovz[frame], ovz[frame]]
        )

        # Misalignment
        mis_line.set_data(
            -t[:frame+1].to_value(u.Gyr),
            misalignment_smooth[:frame+1]
        )

        angle = misalignment_smooth[frame]

        # angle_text.set_text(
        #  fontsize=13
        # ) 

        fig.suptitle(
        rf"Halo: {halo_label}  |  "
        rf"Mass: {mass_label}  |  "
        rf"Lookback time = {(-t[frame].to_value(u.Gyr)):.2f} Gyr  |  "
        rf"$\theta_{{\rm mis}} = {angle:.2f}^\circ$",
        fontsize=16,
        y=0.97
        )

        return sc_x, sc_y, sc_z, mis_line

    ani = FuncAnimation(fig, update, frames=len(t), interval=100)

    writer = FFMpegWriter(fps=6, codec="libx264", bitrate=2500)

    ani.save(filename, writer=writer, dpi=140)
    plt.close(fig)

    print(f"Saved animation to {filename}")

In [ ]:
for i, s in enumerate(streams):

    print(f"\nProcessing stream {i} ({s['halo']})")

    # ----------------------------------
    # 1. Build Hamiltonian
    # ----------------------------------
    H = make_galactic_hamiltonian(q=s["q"])

    # ----------------------------------
    # 2. Extract present-day stream snapshot
    # ----------------------------------
    stream_snapshot = extract_stream_snapshot(
        s["stream"],
        s["t"],
        time_index=0     # <-- FIXED
    )

    n_particles = stream_snapshot.pos.x.shape[0]
    print(f"Particles: {n_particles}")

    # ----------------------------------
    # 3. Subsample particles
    # ----------------------------------
    n_viz = min(3000, n_particles)
    idx = np.random.choice(n_particles, n_viz, replace=False)
    stream_small = stream_snapshot[idx]

    # ----------------------------------
    # 4. Time grid: present → 4 Gyr ago
    # ----------------------------------
    # t_anim = np.arange(
    #     0,
    #     -4000,
    #     -20
    # ) * u.Myr

    t_anim = np.arange(-4000, 0, 20) * u.Myr

    # ----------------------------------
    # 5. Integrate stream particles
    # ----------------------------------
    stream_orbits = H.integrate_orbit(
        stream_small,
        t=t_anim,
        Integrator=LeapfrogIntegrator
    )

    # ----------------------------------
    # 6. Use correct present-day progenitor
    # ----------------------------------
    prog_present = s["orbit"][0]   # <-- FIXED

    orbit_anim = H.integrate_orbit(
        prog_present,
        t=t_anim,
        Integrator=LeapfrogIntegrator
    )

    # ----------------------------------
    # 7. Sanity diagnostics
    # ----------------------------------
    print("Orbit present:", s["orbit"][0].pos.xyz)
    print("Stream present mean:",
          stream_snapshot.pos.xyz.mean(axis=1))

    # ----------------------------------
    # 8. Animate
    # ----------------------------------
    prog_mass = int(s['mass'].value)
    
    animate_full_phase_space(
        stream_orbits=stream_orbits,
        orbit=orbit_anim,
        t=t_anim,
        halo_label=s["halo"],
        mass_label=prog_mass,
        filename=f"../figures/all_axis_phase_space_{prog_mass}_{s['halo']}.mp4",
        n_stream_viz=n_viz
    )


Processing stream 0 (spherical)
Particles: 3000
Orbit present: [8.5 0.  5. ] kpc
Stream present mean: [8.50040161e+00 1.84213331e-04 4.99949237e+00] kpc


/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:99: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_x = ax_x.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:100: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_y = ax_y.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:101: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_z = ax_z.scatter([], [], s=6, cmap="coolwarm", norm=norm)


Saved animation to ../figures/all_axis_phase_space_8000_spherical.mp4

Processing stream 1 (spherical)
Particles: 3000
Orbit present: [8.5 0.  5. ] kpc
Stream present mean: [8.49984327e+00 2.11045332e-05 4.99982258e+00] kpc


/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:99: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_x = ax_x.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:100: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_y = ax_y.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:101: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_z = ax_z.scatter([], [], s=6, cmap="coolwarm", norm=norm)


Saved animation to ../figures/all_axis_phase_space_10000_spherical.mp4

Processing stream 2 (spherical)
Particles: 3000
Orbit present: [8.5 0.  5. ] kpc
Stream present mean: [8.50085084e+00 4.76084264e-05 5.00030825e+00] kpc


...

/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:99: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_x = ax_x.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:100: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_y = ax_y.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:101: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_z = ax_z.scatter([], [], s=6, cmap="coolwarm", norm=norm)


Saved animation to ../figures/all_axis_phase_space_20000_oblate.mp4

Processing stream 6 (prolate)
Particles: 3000
Orbit present: [8.5 0.  5. ] kpc
Stream present mean: [8.50061226e+00 2.65886544e-05 5.00025279e+00] kpc


/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:99: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_x = ax_x.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:100: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_y = ax_y.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:101: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_z = ax_z.scatter([], [], s=6, cmap="coolwarm", norm=norm)


Saved animation to ../figures/all_axis_phase_space_8000_prolate.mp4

Processing stream 7 (prolate)
Particles: 3000
Orbit present: [8.5 0.  5. ] kpc
Stream present mean: [8.49994863e+00 1.27577847e-04 4.99945462e+00] kpc


/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:99: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_x = ax_x.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:100: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_y = ax_y.scatter([], [], s=6, cmap="coolwarm", norm=norm)
/var/folders/qb/fmhr_ygx4j52gp053y5rtyp80000gn/T/ipykernel_2085/3366680628.py:101: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  sc_z = ax_z.scatter([], [], s=6, cmap="coolwarm", norm=norm)
